# SUM Experiments Using Synthetic Dataset

In [1]:
import sys
from os import path
sys.path.insert(0, '../src')
import one_to_n
sys.path.insert(0, '../tests')
import create_synthetic_dataset
import datetime
import textdistance
import editdistance
import pandas as pd
import networkx as nx
import re

import editdistance
from Matching import core2
from Matching import analyze
from Matching import matcher
import sys
import os
import datetime

### Create Synthetic Dataset

In [2]:
# Table A later gets duplicated in the bipartite graph maker function.
table_a_non_duplicated = create_synthetic_dataset.create_first_df(100)

table_b_non_typo = create_synthetic_dataset.create_second_df(100)

table_b = create_synthetic_dataset.add_typo(table_a_non_duplicated, table_b_non_typo)

print(table_a_non_duplicated)
print(table_b)

table_a_non_duplicated.to_csv(r's_data1.csv', index = False, header=True)
table_b.to_csv(r's_data2.csv', index = False, header=True)

                name age
0      Nicholas Chen   1
1   Jacqueline Smith   7
2      Mary Gonzalez   0
3      Mark Thompson   6
4       Donald Smith   0
..               ...  ..
95     Brittany Hill   0
96      William Kent   9
97  Raymond Marshall   8
98      Michael Webb   3
99        Julie Cook   7

[100 rows x 2 columns]
                   name  age
0        Nicholas Chenb    1
1       Nicholas ChenUQ    5
2   Jacqueline SmithNwc    6
3      Mary Gonzalezmut    3
4      Mark ThompsonDKF    4
..                  ...  ...
95    Jessica PalmerAvY    0
96     Brittany Hilldxm    8
97      William KentXie    7
98  Raymond MarshalldmG    7
99      Michael WebbqCE    0

[100 rows x 2 columns]


### 1. Bipartite Matching

In [3]:
now = datetime.datetime.now()
bipartite_graph_result = one_to_n.keycomp_treshold_updated_maximal_construct_graph(table_a_non_duplicated, table_b, "name", 0.09, 3)
timing_tresh = (datetime.datetime.now()-now).total_seconds()
print("---- Timing for Graph Construction with Treshold Constraint ----")
print(timing_tresh,"seconds")

---- Timing for Graph Construction with Treshold Constraint ----
0.236002 seconds


In [4]:
def SUM_edit_edge_weight(bip_graph):
    for u,v,d in bip_graph.edges(data=True):
        val_tuple_1 = u.split("_")
        val_tuple_2 = v.split("_")
        
        if len(val_tuple_1) == 4:
            val1 = re.sub("[^0-9]", "", val_tuple_1[2])
        else: 
            val1 = re.sub("[^0-9]", "", val_tuple_1[1])
            
        if len(val_tuple_2) == 4:
            val2 =re.sub("[^0-9]", "", val_tuple_2[2])
        else:
            val2 =re.sub("[^0-9]", "", val_tuple_2[1])

        d['weight'] = float(val1) + float(val2)

    return bip_graph

sum_weighted_graph = SUM_edit_edge_weight(bipartite_graph_result)

## Maximal Matching

In [5]:
print("\n\n 'SUM' MAXIMAL MATCHING:")
now = datetime.datetime.now()
matching_set_maximal = nx.algorithms.matching.max_weight_matching(sum_weighted_graph)
timing_match = (datetime.datetime.now()-now).total_seconds()
print("The Maximal Matching Set is:", matching_set_maximal, "\n")
print("---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----")
print(timing_match,"seconds")
print("The number of edges in the graph is:", sum_weighted_graph.number_of_edges(), "\n")


# print("The Maximal Matching Set is:", matching_set_maximal, "\n")



 'SUM' MAXIMAL MATCHING:
The Maximal Matching Set is: {('Emily Harris_0_(8,)_1', 'Lisa ChanpVD_(6,)_2'), ('Lauren JenningsyeV_(0,)_2', 'Lauren Jennings_0_(5,)_1'), ('Anne Bates_2_(6,)_1', 'Donna JonesJJu_(8,)_2'), ('Jacqueline Smith_0_(7,)_1', 'Jacqueline SmithNwc_(6,)_2'), ('Steven Webb_2_(4,)_1', 'Steven CarrollJvB_(5,)_2'), ('Steven Lopez_2_(4,)_1', 'Steven LopezpDQ_(3,)_2'), ('Ariel Turner_0_(9,)_1', 'James TatewWY_(3,)_2'), ('James JohnsonmXJ_(2,)_2', 'James Tate_2_(7,)_1'), ('Lawrence Houston_1_(8,)_1', 'Lauren BarrettBoH_(8,)_2'), ('Nicholas AndersonNIP_(2,)_2', 'Nicholas Anderson_0_(4,)_1'), ('Amanda LongTxy_(6,)_2', 'Emily Morton_2_(9,)_1'), ('Megan JohnsonApI_(2,)_2', 'Brian Wilson_0_(7,)_1'), ('Hector Coleman_2_(4,)_1', 'Hector ColemanNDZ_(4,)_2'), ('Cindy CoxbNU_(1,)_2', 'Lisa Chan_0_(9,)_1'), ('Daniel ChungqcI_(4,)_2', 'Gabriel Bennett_1_(9,)_1'), ('Amber Figueroa_2_(1,)_1', 'Amber FigueroaDwS_(9,)_2'), ('Andrea BrooksXtN_(2,)_2', 'Adam Thompson_1_(8,)_1'), ('Cesar Nelso

## Minimal Matching

In [6]:
def minimal_matching(sum_weighted_graph):

    new_graph = sum_weighted_graph.copy()
    max_weight = max([d['weight'] for u,v,d in new_graph.edges(data=True)])
    for u,v,d in new_graph.edges(data=True):
        d['weight'] = max_weight - d['weight']

    matching_set_minimal = nx.algorithms.matching.max_weight_matching(new_graph)
    return matching_set_minimal

In [7]:
print("\n\n 'SUM' MINIMAL MATCHING RESULTS:")
print(nx.bipartite.is_bipartite(sum_weighted_graph))
now = datetime.datetime.now()
matching_set_minimal = minimal_matching(sum_weighted_graph)
timing_match = (datetime.datetime.now()-now).total_seconds()
print("The Minimal Matching Set is:", matching_set_minimal, "\n")
print("---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----")
print(timing_match,"seconds")
print("The number of edges in the graph is:", sum_weighted_graph.number_of_edges(), "\n")



 'SUM' MINIMAL MATCHING RESULTS:
True
The Minimal Matching Set is: {('James TatewWY_(3,)_2', 'Amber Figueroa_2_(1,)_1'), ('Colleen OwensaTG_(8,)_2', 'Cody Hensley_1_(2,)_1'), ('Hector Coleman_2_(4,)_1', 'Hector ColemanNDZ_(4,)_2'), ('Donald Smith_0_(0,)_1', 'Donald SmithWHo_(9,)_2'), ('Philip VelasquezaqC_(6,)_2', 'Philip Velasquez_2_(0,)_1'), ('Whitney Jenkins_2_(1,)_1', 'Christine JenningsiRx_(1,)_2'), ('Caitlin Tucker_1_(0,)_1', 'Caitlin TuckerEfY_(7,)_2'), ('Dr. Laura PetersPTG_(4,)_2', 'Dr. Laura Peters_2_(6,)_1'), ('Jill Sellers_2_(0,)_1', 'Jill SellersNgD_(9,)_2'), ('Jill Sellers_1_(0,)_1', 'Seth Wheelerqjz_(3,)_2'), ('Jackie Chase DVM_0_(7,)_1', 'Jackie Chase DVMyVW_(9,)_2'), ('Jill Sellers_0_(0,)_1', 'William KentXie_(7,)_2'), ('Chloe DavisijA_(6,)_2', 'Kelly Hawkins_1_(4,)_1'), ('Jessica PalmerAvY_(0,)_2', 'Eric Hale_2_(1,)_1'), ('Whitney JenkinsPPe_(2,)_2', 'Whitney Jenkins_1_(1,)_1'), ('Christopher WatersMOD_(1,)_2', 'Christopher Waters_2_(7,)_1'), ('Walter Armstrong_2_(0

In [8]:
def fetch_sum(bip_graph, matching):
    output = []
    for u,v,d in bip_graph.edges(data=True):
        l = (u, v)
        k = (v, u)
        if l in matching:
            output.append([u,v, d['weight']])
        if k in matching:
            output.append([v,u, d['weight']])
    return output

out_max = fetch_sum(sum_weighted_graph, matching_set_maximal)
out_min = fetch_sum(sum_weighted_graph, matching_set_minimal)

print("\n MAX MATCHES WITH SUM WEIGHTS")
for val in out_max:
    print(val)
    
print("\n MIN MATCHES WITH SUM WEIGHTS")
for val2 in out_min:
    print(val2)

def formatted_output(out_max, out_min):
    out_dict = {}
    for (val1,val2, weight) in out_min:
        splitted1 = val1.split("_")
        splitted2 = val2.split("_")
        if len(splitted1) == 4:
            if splitted1[0] in out_dict:
                out_dict[splitted1[0]].append((splitted2[0], "min", weight))
            else:
                out_dict[splitted1[0]] = [(splitted2[0], "min", weight)]

        if len(splitted2) == 4:
            if splitted2[0] in out_dict:
                out_dict[splitted2[0]].append((splitted1[0], "min", weight))
            else:
                out_dict[splitted2[0]] = [(splitted1[0], "min", weight)]
            
    for (val1,val2, weight) in out_max:
        splitted1 = val1.split("_")
        splitted2 = val2.split("_")
        if len(splitted1) == 4:
            if splitted1[0] in out_dict:
                out_dict[splitted1[0]].append((splitted2[0], "max", weight))
            else:
                out_dict[splitted1[0]] = [(splitted2[0], "max", weight)]

        if len(splitted2) == 4:
            if splitted2[0] in out_dict:
                out_dict[splitted2[0]].append((splitted1[0], "max", weight))
            else:
                out_dict[splitted2[0]] = [(splitted1[0], "max", weight)]
    return out_dict
            
    
form_output = formatted_output(out_max,out_min)

# print("\n FORMAL OUTPUT")
# for i,val in form_output.items():
#     print(i, val)


 MAX MATCHES WITH SUM WEIGHTS
['Chloe Davis_0_(9,)_1', 'Nicholas Chenb_(1,)_2', 10.0]
['William Kent_1_(9,)_1', 'Nicholas ChenUQ_(5,)_2', 14.0]
['Eric HalecSX_(7,)_2', 'Chloe Davis_1_(9,)_1', 16.0]
['Emily Harris_0_(8,)_1', 'Lisa ChanpVD_(6,)_2', 14.0]
['Nicholas AndersonNIP_(2,)_2', 'Nicholas Anderson_0_(4,)_1', 6.0]
['Jacqueline Smith_0_(7,)_1', 'Jacqueline SmithNwc_(6,)_2', 13.0]
['Jacqueline Smith_2_(7,)_1', 'Jacqueline BakeraJz_(9,)_2', 16.0]
['Mary Gonzalez_0_(0,)_1', 'Mary Gonzalezmut_(3,)_2', 3.0]
['Lauren Barrett_0_(9,)_1', 'Laura GrossFtV_(0,)_2', 9.0]
['Amanda LongTxy_(6,)_2', 'Emily Morton_2_(9,)_1', 15.0]
['Carla GarciahBa_(5,)_2', 'Chelsea Garcia_0_(8,)_1', 13.0]
['Julie Cook_1_(7,)_1', 'Tracy CoxSiS_(7,)_2', 14.0]
['Adam Thompson_2_(8,)_1', 'Mark ThompsonDKF_(4,)_2', 12.0]
['Adam Thompson_0_(8,)_1', 'Adam ThompsonHnJ_(2,)_2', 10.0]
['Donald Wilson_2_(5,)_1', 'Donald SmithWHo_(9,)_2', 14.0]
['Cesar Nelson_2_(7,)_1', 'Donald WilsonTrD_(6,)_2', 13.0]
['Anne Bates_2_(6,)_1'

## BP Matching: Sum row of Min and Max

In [9]:
def sum_total_weights(max_min_list):
    if max_min_list == [] or max_min_list == None:
        print("ERROR: NO SIMILARITY FOUND IN NAIVE OR RANDOM SAMPLING APPROACH. Suggestion: Decrease Similarity Matching Threshold.")
        return None
    total = 0
    for i in max_min_list:
        total += i[-1]
    return total

total_max = sum_total_weights(out_max)
print("BP Matching: Highest bound for maximum:", total_max)

total_min = sum_total_weights(out_min)
print("BP Matching: Lowest bound for minimum:", total_min)

BP Matching: Highest bound for maximum: 1041.0
BP Matching: Lowest bound for minimum: 619.0


### Prepare Data for Comparison Experimentation

In [10]:
# print(os.getcwd())
sample_size = 50
table_a_dup = one_to_n.create_duplicates(table_a_non_duplicated, "name", 3)
table_a_dup.to_csv(r's_data1_dup.csv', index = False, header=True)
cat_table1_dup = core2.data_catalog('s_data1_dup.csv')
cat_table1 = core2.data_catalog('s_data1.csv')
cat_table2 = core2.data_catalog('s_data2.csv')
print('Loaded catalogs.')

Loaded catalogs.


### 2. Naive Matching Performance Evaluation

In [11]:
# MAX MATCHING
print("NAIVE MAX MATCHING")
print('Performing compare all match (edit distance)...')
now = datetime.datetime.now()
max_compare_all_edit_match = matcher.matcher_dup_updated(3, cat_table1_dup,cat_table2,editdistance.eval, matcher.all, 0.09)
naive_time_edit = (datetime.datetime.now()-now).total_seconds()
print("Naive Edit Distance Matching computation time taken: ", naive_time_edit, " seconds")
#print('Compare All Matcher (Edit Distance) Performance: ' + str(core.eval_matching(compare_all_edit_match)))


print('Performing compare all match (jaccard distance)...')
now = datetime.datetime.now()
max_compare_all_jaccard_match = matcher.matcher_dup_updated(3, cat_table1_dup,cat_table2,analyze.jaccard_calc, matcher.all, 0.09)
naive_time_jaccard = (datetime.datetime.now()-now).total_seconds()
print("Naive Jaccard Matching computation time taken: ", naive_time_jaccard, " seconds", "\n")
#print('Compare All Matcher (Jaccard Distance) Performance: ' + str(core2.eval_matching(compare_all_jaccard_match)))

# MIN MATCHING
print("NAIVE MIN MATCHING")
print('Performing compare all match (edit distance)...')
now = datetime.datetime.now()
min_compare_all_edit_match = matcher.matcher_updated(3, cat_table1,cat_table2,editdistance.eval, matcher.all, 0.09)
naive_time_edit = (datetime.datetime.now()-now).total_seconds()
print("Naive Edit Distance Matching computation time taken: ", naive_time_edit, " seconds")
#print('Compare All Matcher (Edit Distance) Performance: ' + str(core.eval_matching(compare_all_edit_match)))


print('Performing compare all match (jaccard distance)...')
now = datetime.datetime.now()
min_compare_all_jaccard_match = matcher.matcher_updated(3, cat_table1,cat_table2,analyze.jaccard_calc, matcher.all, 0.09)
naive_time_jaccard = (datetime.datetime.now()-now).total_seconds()
print("Naive Jaccard Matching computation time taken: ", naive_time_jaccard, " seconds")
#print('Compare All Matcher (Jaccard Distance) Performance: ' + str(core2.eval_matching(compare_all_jaccard_match)))

NAIVE MAX MATCHING
Performing compare all match (edit distance)...
Naive Edit Distance Matching computation time taken:  0.194388  seconds
Performing compare all match (jaccard distance)...
Naive Jaccard Matching computation time taken:  0.129257  seconds 

NAIVE MIN MATCHING
Performing compare all match (edit distance)...
Naive Edit Distance Matching computation time taken:  0.043087  seconds
Performing compare all match (jaccard distance)...
Naive Jaccard Matching computation time taken:  0.007319  seconds


### 3. Random Sampling Matching Performance Evaluation

In [12]:
# MAX MATCHING
print("RANDOM SAMPLE MAX MATCHING")
print('Performing random sample match (edit distance)...')
now = datetime.datetime.now()
max_compare_sampled_edit_match = matcher.matcher_dup_updated(3, cat_table1_dup,cat_table2,editdistance.eval, matcher.random_sample, 0.09, sample_size)
sim_time_edit = (datetime.datetime.now()-now).total_seconds()
print("Simulation-Based Edit Distance Matching computation time taken: ", sim_time_edit, " seconds")
#print('Random Sample Matcher (Edit Distance) Performance: ' + str(core.eval_matching(compare_all_edit_match)))

# print('Performing random sample match (jaccard distance)...')
# now = datetime.datetime.now()
# max_compare_sampled_jaccard_match = matcher.matcher_dup_updated(cat_table1_dup,cat_table2,analyze.jaccard_calc, matcher.random_sample, 0.05, sample_size)
# sim_time_jaccard = (datetime.datetime.now()-now).total_seconds()
# print("Simulation-Based Jaccard Matching computation time taken: ", sim_time_jaccard, " seconds", "\n")
#print('Random Sample Matcher (Jaccard Distance) Performance: ' + str(core.eval_matching(compare_all_jaccard_match)))


# MIN MATCHING
print("RANDOM SAMPLE MIN MATCHING")
print('Performing random sample match (edit distance)...')
now = datetime.datetime.now()
min_compare_sampled_edit_match = matcher.matcher_updated(3, cat_table1,cat_table2,editdistance.eval, matcher.random_sample, 0.09, sample_size)
sim_time_edit = (datetime.datetime.now()-now).total_seconds()
print("Simulation-Based Edit Distance Matching computation time taken: ", sim_time_edit, " seconds")
#print('Random Sample Matcher (Edit Distance) Performance: ' + str(core.eval_matching(compare_all_edit_match)))

# print('Performing random sample match (jaccard distance)...')
# now = datetime.datetime.now()
# min_compare_sampled_jaccard_match = matcher.matcher_updated(cat_table1,cat_table2,analyze.jaccard_calc, matcher.random_sample, 0.5, sample_size)
# sim_time_jaccard = (datetime.datetime.now()-now).total_seconds()
# print("Simulation-Based Jaccard Matching computation time taken: ", sim_time_jaccard, " seconds")
#print('Random Sample Matcher (Jaccard Distance) Performance: ' + str(core.eval_matching(compare_all_jaccard_match)))

RANDOM SAMPLE MAX MATCHING
Performing random sample match (edit distance)...
Simulation-Based Edit Distance Matching computation time taken:  0.120959  seconds
RANDOM SAMPLE MIN MATCHING
Performing random sample match (edit distance)...
Simulation-Based Edit Distance Matching computation time taken:  0.047992  seconds


In [13]:
print(max_compare_all_jaccard_match)

naive_total_max = sum_total_weights(max_compare_all_edit_match)
sampled_total_max = sum_total_weights(max_compare_sampled_edit_match)
bp_total_max = sum_total_weights(out_max)


print("MAX Matching Bound:")
print("BP Matching: ", total_max)
print("NAIVE Matching: ", naive_total_max)
print("SAMPLED Matching: ", sampled_total_max, "\n")

naive_total_min = sum_total_weights(min_compare_all_edit_match)
sampled_total_min = sum_total_weights(min_compare_sampled_edit_match)
bp_total_min = sum_total_weights(out_min)


print("MIN Matching Bound:")
print("BP Matching: ", total_min)
print("NAIVE Matching: ", naive_total_min)
print("SAMPLED Matching: ", sampled_total_min, "\n")

[('Nicholas Chen_0', 'Nicholas Chenb', 2), ('Nicholas Chen_0', 'Nicholas ChenUQ', 6), ('Nicholas Chen_0', 'Jacqueline SmithNwc', 7), ('Nicholas Chen_0', 'Mary Gonzalezmut', 4), ('Nicholas Chen_0', 'Mark ThompsonDKF', 5), ('Nicholas Chen_0', 'Donald SmithWHo', 10), ('Nicholas Chen_0', 'Seth Wheelerqjz', 4), ('Nicholas Chen_0', 'Melissa VargasNqa', 9), ('Nicholas Chen_0', 'Cesar NelsondSU', 5), ('Nicholas Chen_0', 'Laura GrossFtV', 1), ('Nicholas Chen_0', 'Alexander DavisRbV', 9), ('Nicholas Chen_0', 'Eric KimdyG', 1), ('Nicholas Chen_0', 'Chloe DavisijA', 7), ('Nicholas Chen_0', 'Dr. Megan McbrideNnt', 3), ('Nicholas Chen_0', 'Donald WilsonTrD', 7), ('Nicholas Chen_0', 'Matthew Johnsonjxr', 1), ('Nicholas Chen_0', 'Lawrence HoustonXsb', 6), ('Nicholas Chen_0', 'Christine JenningsiRx', 2), ('Nicholas Chen_0', 'Whitney JenkinsPPe', 3), ('Nicholas Chen_0', 'Andrea BrooksXtN', 3), ('Nicholas Chen_0', 'Jacob WeberHwP', 1), ('Nicholas Chen_0', 'Rachel OrtegaDAz', 1), ('Nicholas Chen_0', 'Trac

In [14]:
# print(max_compare_all_edit_match)

# print(table_a_non_duplicated)
# print(table_b)
# print(min_compare_all_edit_match)

## Formal Output of Experiment Matches (Naive and Random Sampling)

In [15]:
def formatted_output_other(out_max, out_min):
    out_dict = {}
    for (val1,val2, weight) in out_min:
        splitted1 = val1.split("_")
        if splitted1[0] in out_dict:
            out_dict[splitted1[0]].append((val2, "min", weight))
        else:
            out_dict[splitted1[0]] = [(val2, "min", weight)]

    for (val1,val2, weight) in out_max:
        splitted1 = val1.split("_")
        if splitted1[0] in out_dict:
            out_dict[splitted1[0]].append((val2, "min", weight))
        else:
            out_dict[splitted1[0]] = [(val2, "min", weight)]

    return out_dict

In [16]:
form_output_all = formatted_output_other(max_compare_all_edit_match,min_compare_all_edit_match)
form_output_sampled = formatted_output_other(max_compare_sampled_edit_match,min_compare_sampled_edit_match)

In [17]:
# for i in form_output_all.items():
#     print(i, "\n\n")

## Results for "COUNT"

In [18]:
# Count all the matched people whose age is above 30.
def count_filter_condition(filter_condition, bip_graph):
    record_edges_to_delete = []
    
    for u,v,d in bip_graph.edges(data=True):
        val_tuple_1 = u.split("_")
        val_tuple_2 = v.split("_")
        
        if len(val_tuple_1) == 4:
            val1 = re.sub("[^0-9]", "", val_tuple_1[2])
        else: 
            val1 = re.sub("[^0-9]", "", val_tuple_1[1])
            
        if len(val_tuple_2) == 4:
            val2 =re.sub("[^0-9]", "", val_tuple_2[2])
        else:
            val2 =re.sub("[^0-9]", "", val_tuple_2[1])
            
        sum_vals = float(val1) + float(val2)
            
        if float(val1) < filter_condition or float(val2) < filter_condition or sum_vals < filter_condition:
            # record_edges_to_delete.append((u,v))
            d['weight'] = 0
        else:
            d['weight'] = sum_vals
            
    for i,j in record_edges_to_delete:
        bip_graph.remove_edge(i,j)

    return bip_graph

In [19]:
count_edited_bip_graph = count_filter_condition(3, bipartite_graph_result)
#count_edited_bip_graph = one_to_n.COUNT_edit_edge_weight(bipartite_graph_result)

# Maximum matching for Count
count_matching_set_maximal = nx.algorithms.matching.max_weight_matching(count_edited_bip_graph)

# Minimum matching for Count
count_matching_set_minimal = minimal_matching(count_edited_bip_graph)

out_max_count = fetch_sum(count_edited_bip_graph, count_matching_set_maximal)
out_min_count = fetch_sum(count_edited_bip_graph, count_matching_set_minimal)
form_output_count = formatted_output(out_max_count,out_min_count)

# print(form_output_count)
for i in form_output_count.items():
    print(i)

('Tara Owens', [('Teresa Petersenf', 'min', 9.0), ('Travis FowlerUaJ', 'min', 8.0), ('Stanley ObrienwSc', 'min', 13.0)])
('Peter Jensen', [('Teresa PetersenBt', 'min', 0), ('Spencer HensleyMij', 'min', 15.0)])
('Adam Mcpherson', [('Sara SchaeferscT', 'min', 0), ('Adam McphersonwnT', 'min', 0)])
('Elizabeth Singh', [('Erica PetersonkRf', 'min', 13.0), ('Elizabeth Taylorjbl', 'max', 12.0)])
('Lisa Rivera MD', [('Tara OwensgYD', 'min', 0), ('Lisa Rivera MDsIX', 'min', 0)])
('Wanda Herrera', [('Angela WeaverjIG', 'min', 6.0), ('Kevin Harrisgaq', 'min', 12.0)])
('Sheila Riley', [('Lisa RiveraEWT', 'min', 0)])
('Billy Williams', [('Sharon WillisEnL', 'min', 0), ('Billy WilliamsYnw', 'min', 0)])
('Kathryn Gill', [('Kathryn GillFrd', 'min', 0), ('Peter JensenXll', 'min', 0)])
('Terry Chan', [('Karen CraigwBx', 'min', 0), ('Tim ChapmanURu', 'min', 0)])
('Donna Klein', [('Diana WestpJe', 'min', 0)])
('Caitlin Turner', [('Brian WarnertNc', 'min', 0), ('David DunnckC', 'max', 14.0), ('Caitlin Turn

In [20]:
def experiment_filter_count(filter_condition, matching_list):
    to_keep_list = []
    for (i,j,k) in matching_list:
        if int(k) >= filter_condition:
            to_keep_list.append((i,j,k))
        else:
            continue
    return to_keep_list

# create a dummy edge that has 0 weight
        

In [21]:
total_max_count = sum_total_weights(out_max_count)
print("BP Matching: Highest bound for maximum:", total_max_count)

total_min_count = sum_total_weights(out_min_count)
print("BP Matching: Lowest bound for minimum:", total_min_count, "\n")


# Apply predicate constraint to count for min matching
filtered_naive_max = experiment_filter_count(3, min_compare_all_edit_match)
filtered_sampled_max = experiment_filter_count(3, min_compare_sampled_edit_match)

naive_total_max_count = sum_total_weights(filtered_naive_max)
sampled_total_max_count = sum_total_weights(filtered_sampled_max)

print("MAX Count of Matches:")
print("BP Matching Count:", total_max_count)
print("NAIVE Matching Count: ", naive_total_max_count)
print("SAMPLED Matching Count:", sampled_total_max_count, "\n")

# Apply predicate constraint to count for max matching
filtered_naive_min = experiment_filter_count(3, max_compare_all_edit_match)
filtered_sampled_min = experiment_filter_count(3, max_compare_sampled_edit_match)

naive_total_min_count = sum_total_weights(filtered_naive_min)
sampled_total_min_count = sum_total_weights(filtered_sampled_min)

print("MIN Count of Matches:")
print("BP Matching Count:", total_min_count)
print("NAIVE Matching Count: ", naive_total_min_count)
print("SAMPLED Matching Count:", sampled_total_min_count, "\n")

BP Matching: Highest bound for maximum: 802.0
BP Matching: Lowest bound for minimum: 258.0 

MAX Count of Matches:
BP Matching Count: 802.0
NAIVE Matching Count:  2731
SAMPLED Matching Count: 1749 

MIN Count of Matches:
BP Matching Count: 258.0
NAIVE Matching Count:  12186
SAMPLED Matching Count: 6170 



### Filter Naive Matching Results

In [22]:
data_lookup = matcher.create_lookup(cat_table1_dup,cat_table2, 'name', 'age')

max_all_filtered_res = matcher.filter_results(data_lookup, 15.0, max_compare_all_edit_match)
min_all_filtered_res = matcher.filter_results(data_lookup, 15.0, min_compare_all_edit_match)

max_sampled_filtered_res = matcher.filter_results(data_lookup, 15.0, max_compare_sampled_edit_match)
min_sampled_filtered_res = matcher.filter_results(data_lookup, 15.0, min_compare_sampled_edit_match)

naive_total_max_count = sum_total_weights(filtered_naive_max)
sampled_total_max_count = sum_total_weights(filtered_sampled_max)

In [23]:
print("Naive Max Matching for COUNT, Filtered According to Age Threshold: ",len(max_all_filtered_res))
print("Naive Min Matching for COUNT, Filtered According to Age Threshold: ",len(min_all_filtered_res))

print("Random Sampling Max Matching for COUNT, Filtered According to Age Threshold: ", len(max_sampled_filtered_res))
print("Random Sampling Min Matching for COUNT, Filtered According to Age Threshold: ", len(min_sampled_filtered_res))

Naive Max Matching for COUNT, Filtered According to Age Threshold:  1425
Naive Min Matching for COUNT, Filtered According to Age Threshold:  321
Random Sampling Max Matching for COUNT, Filtered According to Age Threshold:  720
Random Sampling Min Matching for COUNT, Filtered According to Age Threshold:  211
